In [40]:
import sklearn as sk
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import os
from sklearn import metrics

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler


In [22]:
num_clusts = range(40)


In [37]:
X = pd.read_csv(os.getcwd() + '/top5_chars.csv')
X.index = X.iloc[:,0]
X = X.dropna()

X.drop('Unnamed: 0',axis=1,inplace=True)
X.drop(['cursewords','FK','0','tot_sents'],axis=1,inplace=True)

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [14]:
#so, we will actually have to break these guys down into senteces... 
# so I will have to do the sentence tokenizer :) 
def jacardian_distance(document_1_data, document_2_data):

    
    #for key in processed_article_hash.keys(): 
    #art_uniq_ws[key] = []
    #for sent in processed_article_hash[key]:
    #    art_uniq_ws[key].extend(sent)
    #    art_uniq_ws[key] = list(set(art_uniq_ws[key]))
    d1_uniqs = []
    for sent in document_1_data:
        d1_uniqs.extend(sent)
        d1_uniqs = list(set(d1_uniqs))
    d2_uniqs = []
    for sent in document_2_data:
        d2_uniqs.extend(sent)
        d2_uniqs = list(set(d2_uniqs))
        
    words_in_doc_1_not_in_doc_2 = [value for value in d1_uniqs if value not in d2_uniqs]
    words_in_doc_2_not_in_doc_1 = [value for value in d2_uniqs if value not in d1_uniqs]
    words_in_both_doc_1_and_doc_2 = [value for value in d1_uniqs if value in d2_uniqs]
    
    
    
    #overlap = [value for value in d1_uniqs if value in d2_uniqs] 
    #in1Not2 = [value for value in d1_uniqs if value not in d2_uniqs]
    #in2Not1 = [value for value in d2_uniqs if value not in d1_uniqs]
   # print(len(in1Not2))
    #print(len(in2Not1))
    #print(len(sv))
   # print(len(sv)+len(in2Not1)+len(in1Not2))

   # sv = d2_uniqs + d1_uniqs
   # sv = list(set(sv))
   # print(len(overlap))
    
    
    
    jacardian = len(words_in_both_doc_1_and_doc_2)/(len(words_in_both_doc_1_and_doc_2) + len(words_in_doc_1_not_in_doc_2) + len(words_in_doc_2_not_in_doc_1)) # divide the counts appropiately
    
    return jacardian
    
    
    
#Cosine Similarity: Here we want to create vector representations for each document. Specifically, we want to come up with a vector that is based on the list of all words that occur across both documents. Then for each document we will create a vector that includes the counts of the number of time a word occurs in the document.

#So if the document 1 is: "the ship sails at midnight" and document 2 is: "the crow flies at noon." We would be creating a vector like: [the, ship, sails, at, midnight, crow, flies, noon]. Then we would calculate the values of the vector for each document. For document 1: [1,1,1,1,1,0,0,0] and for document 2: [1,0,0,1,0,1,1,1]. With these two vectors we would simply take the dot product and that would provide the cosine similarity. 

def cosine_similarity(document_1_data, document_2_data):
    document_vector_word_index = [] # here fill this with an ordered list of all the unique words across both documents
    d1_uniqs = []
    d1_nun = []
    for sent in document_1_data:
        d1_uniqs.extend(sent)
        d1_nun.extend(sent)
        d1_uniqs = list(set(d1_uniqs))
    d2_uniqs = []
    d2_nun = []
    for sent in document_2_data:
        d2_uniqs.extend(sent)
        d2_uniqs = list(set(d2_uniqs))
        d2_nun.extend(sent)

    
    all_words = d1_nun+d2_nun   

    overlap = d1_uniqs + d2_uniqs 
    # so this is all of the unique words across the two documents
    document_vector_word_index += list(set(overlap))
    

    document_1_vector = list(np.zeros(len(document_vector_word_index)))  # fill in the array with the frequency of the words in the document
    document_2_vector = list(np.zeros(len(document_vector_word_index))) # fill in the array with the frequency of the words in the document

    for wOrd in d1_nun:
        document_1_vector[document_vector_word_index.index(wOrd)] +=1
    
    for wOrd in d2_nun:
        document_2_vector[document_vector_word_index.index(wOrd)] +=1
    
    d1_norm = (np.sum(np.array(document_1_vector)**2))**(1/2)
    d2_norm = (np.sum(np.array(document_2_vector)**2))**(1/2)
    
    cosSim = np.dot(document_1_vector,document_2_vector)/(d1_norm*d2_norm)


    
    return (cosSim)
#document_1_vector,document_2_vector,document_vector_word_index, are the things I tried to return in testing
#dot_product_of_two_document_vectors # you can refer to the numpy information on how to calculate the dot product of vectors

In [ ]:
# create a variable to store your table data... you could use a hash or some other data structure. 
# We just want it to identify which document is being compared to which other document.

data_structure_for_jacard_similarity = []
data_structure_for_cosine_similarity = []
# so I think what I'm ACTUALLY going to do here 
# is just append a new column
# and then say the column is .... 
# so we are comparing each character to every OTHER character (including themselves)
for char_1_index in char_series.index:
    for char_2_index in char_series.index:
        # we have the nested for loops as one way to compare each document to each other document
        data_structure_for_jacard_similarity.append(jacardian_distance(char_series[char_1_index], char_series[char_2_index]))
        data_structure_for_cosine_similarity.append(cosine_similarity(char_series[char_1_index], char_series[char_2_index]))

# finally, find some way to present this data back. Either as a straight table or a heatmap.
# so... yeah.... I can probably keep the rows the same as what they were... TECHNICALLY.... 
colz = char_series.index
rowz = char_series.index


for movChar in char_series_index:
    

In [15]:
pd.Series(np.nan,index = char_series.index)

NameError: name 'char_series' is not defined

In [50]:
db = DBSCAN(eps=0.05, min_samples=5).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
#print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
#print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
#print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
#print("Adjusted Rand Index: %0.3f"
#      % metrics.adjusted_rand_score(labels_true, labels))
#print("Adjusted Mutual Information: %0.3f"
#      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

Estimated number of clusters: 7
Estimated number of noise points: 840
Silhouette Coefficient: 0.108


In [57]:
db2 = DBSCAN(eps=0.05, min_samples=7).fit(X)
core_samples_mask2 = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask2[db.core_sample_indices_] = True
labels2 = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_2 = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_2 = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
#print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
#print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
#print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
#print("Adjusted Rand Index: %0.3f"
#      % metrics.adjusted_rand_score(labels_true, labels))
#print("Adjusted Mutual Information: %0.3f"
#      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

Estimated number of clusters: 7
Estimated number of noise points: 840
Silhouette Coefficient: 0.108


ValueError: labels_true must be 1D: shape is (3301, 6)

In [56]:
labels

array([ 0,  0,  1, ...,  1,  1, -1], dtype=int64)

In [61]:
dbz = []
shilz = []


for minp, ep in zip([2,3,4,5,6,7,8,9,10],[.05,.075,.1,.125,.15,.175,.2,.225,.25]):
    db = DBSCAN(eps=ep, min_samples=minp).fit(X)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    dbz.append(db)

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)
#print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
#print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
#print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
#print("Adjusted Rand Index: %0.3f"
#      % metrics.adjusted_rand_score(labels_true, labels))
#print("Adjusted Mutual Information: %0.3f"
#      % metrics.adjusted_mutual_info_score(labels_true, labels))
    print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))
    shilz.append(metrics.silhouette_score(X, labels))
    
    
    

Estimated number of clusters: 80
Estimated number of noise points: 591
Silhouette Coefficient: -0.286
Estimated number of clusters: 9
Estimated number of noise points: 292
Silhouette Coefficient: 0.164
Estimated number of clusters: 3
Estimated number of noise points: 146
Silhouette Coefficient: 0.505
Estimated number of clusters: 2
Estimated number of noise points: 98
Silhouette Coefficient: 0.709
Estimated number of clusters: 2
Estimated number of noise points: 66
Silhouette Coefficient: 0.726
Estimated number of clusters: 2
Estimated number of noise points: 39
Silhouette Coefficient: 0.745
Estimated number of clusters: 2
Estimated number of noise points: 22
Silhouette Coefficient: 0.767
Estimated number of clusters: 2
Estimated number of noise points: 17
Silhouette Coefficient: 0.780
Estimated number of clusters: 1
Estimated number of noise points: 12
Silhouette Coefficient: 0.457


In [64]:
shilz.index(max(shilz))
dbz[7]

7

In [75]:
xlabz1 = pd.Series(dbz[7].labels_)


In [86]:
x = pd.read_csv(os.getcwd() + '/top5_chars.csv')
x['clustering_labels'] = xlabz1
x_1 =  x[x['clustering_labels']==1]['Unnamed: 0']
x_0 = x[x['clustering_labels'] == 0]['Unnamed: 0']
#[['clustering_labels','Unnamed: 0']]
list(x_1)[61:160]

['ANTHONY____Alien-3_script',
 'KYLE____Alien-3_script',
 'RIPLEY____Aliens_script',
 'HICKS____Aliens_script',
 'HUDSON____Aliens_script',
 'BURKE____Aliens_script',
 'STANDARD____Alien_script',
 'ROBY____Alien_script',
 'HUNTER____Alien_script',
 'BROUSSARD____Alien_script',
 'ALI____Ali_script',
 'BUNDINI____Ali_script',
 'CASSIUS____Ali_script',
 'HOWARD COSELL____Ali_script',
 'MARY____All-About-Steve_script',
 'HARTMAN____All-About-Steve_script',
 'HOWARD____All-About-Steve_script',
 "WILLIE____All-the-King's-Men_script",
 "JACK____All-the-King's-Men_script",
 "SADIE____All-the-King's-Men_script",
 "ANNE____All-the-King's-Men_script",
 "STANTON____All-the-King's-Men_script",
 'CARNBY____Alone-in-the-Dark_script',
 'ALINE____Alone-in-the-Dark_script',
 'MOZART____Amadeus_script',
 'SALIERI____Amadeus_script',
 'CONSTANZE____Amadeus_script',
 'JOSEPH____Amadeus_script',
 'OLD SALIERI____Amadeus_script',
 'AMELIA____Amelia_script',
 'GEORGE____Amelia_script',
 'GENE____Amelia_script

In [66]:
dir(db)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_estimator_type',
 '_get_param_names',
 'algorithm',
 'components_',
 'core_sample_indices_',
 'eps',
 'fit',
 'fit_predict',
 'get_params',
 'labels_',
 'leaf_size',
 'metric',
 'metric_params',
 'min_samples',
 'n_jobs',
 'p',
 'set_params']

In [ ]:
core_sample_indices_ : array, shape = [n_core_samples]
Indices of core samples.

components_ : array, shape = [n_core_samples, n_features]
Copy of each core sample found by training.

labels_ : array, shape = [n_samples]
Cluster labels for each point in the dataset given to fit(). Noisy samples are given the labe